# Execution of Renamed PaExec
Detects execution of renamed paexec via imphash and executable product string

## Rule Content
```
- title: Execution of Renamed PaExec
  id: 7b0666ad-3e38-4e3d-9bab-78b06de85f7b
  status: experimental
  description: Detects execution of renamed paexec via imphash and executable product
    string
  references:
  - sha256=01a461ad68d11b5b5096f45eb54df9ba62c5af413fa9eb544eacb598373a26bc
  - https://summit.fireeye.com/content/dam/fireeye-www/summit/cds-2018/presentations/cds18-technical-s05-att&cking-fin7.pdf
  tags:
  - attack.defense_evasion
  - attack.t1036
  - FIN7
  - car.2013-05-009
  date: 2019/04/17
  author: Jason Lynch
  falsepositives:
  - Unknown imphashes
  level: medium
  logsource:
    category: process_creation
    product: windows
    service: null
  detection:
    selection1:
      Product:
      - '*PAExec*'
    selection2:
      Imphash:
      - 11D40A7B7876288F919AB819CC2D9802
      - 6444f8a34e99b8f7d9647de66aabe516
      - dfd6aa3f7b2b1035b76b718f1ddc689f
      - 1a6cca4d5460b1710a12dea39e4a592c
    filter1:
      Image: '*paexec*'
    condition: (selection1 and selection2) and not filter1

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='((file_product.keyword:(*PAExec*) AND hash_imphash:("11D40A7B7876288F919AB819CC2D9802" OR "6444f8a34e99b8f7d9647de66aabe516" OR "dfd6aa3f7b2b1035b76b718f1ddc689f" OR "1a6cca4d5460b1710a12dea39e4a592c")) AND (NOT (process_path.keyword:*paexec*)))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()